# BLASER 2.0: Testing Sentence Similarity

This notebook demonstrates how to use BLASER 2.0 for predicting sentence similarity using the SONAR embedding space. BLASER 2.0 is a family of models for automatic evaluation of machine translation quality based on SONAR embeddings.

There are two main models available:
- **BLASER 2.0 QE (Quality Estimation)**: Predicts similarity between source text and translation without requiring reference translations
- **BLASER 2.0 Ref**: Predicts similarity using source, translation, and reference translation

Both models are based on the [SONAR](https://github.com/facebookresearch/SONAR) (Sentence-level multimOdal and laNguage-Agnostic Representations) framework.

## Installing Required Packages with UV

We'll use `uv` to create a virtual environment and install the necessary packages. UV is a fast Python package installer and resolver, which makes dependency management more efficient.

SONAR requires specific versions of fairseq2 that match the PyTorch and CUDA versions. If you don't have `uv` installed, you can install it first with:
```bash
curl -sSf https://install.python-uv.org | python3
```

In [ ]:
# Create and activate environment with uv using pyproject.toml
!uv venv create .venv
!source .venv/bin/activate

# Install the project and its dependencies from pyproject.toml
!uv pip install -e .

# Check if UV installation was successful
!uv --version

In [ ]:
# Install fairseq2 with the appropriate PyTorch version using our helper script
# This script automatically detects your PyTorch version and installs the compatible fairseq2
!python install_fairseq2.py

# Verify PyTorch version
import torch
print(f"PyTorch version: {torch.__version__}")

# Verify fairseq2 installation
try:
    import fairseq2
    print(f"fairseq2 version: {fairseq2.__version__}")
    print("fairseq2 installation successful!")
except ImportError:
    print("fairseq2 not installed correctly. Please run the helper script again or install manually.")

## Setting Up BLASER 2.0

Now, let's import the necessary modules and set up the BLASER 2.0 models. We'll use both the Quality Estimation (QE) model and the Reference-based model.

In [ ]:
# Import necessary modules
from sonar.inference_pipelines.text import TextToEmbeddingModelPipeline
from sonar.models.blaser.loader import load_blaser_model
import torch

# Set device based on availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load BLASER models
try:
    blaser_ref = load_blaser_model("blaser_2_0_ref").eval()
    blaser_qe = load_blaser_model("blaser_2_0_qe").eval()
    print("BLASER 2.0 models loaded successfully")
    
    # Initialize text embedder
    text_embedder = TextToEmbeddingModelPipeline(
        encoder="text_sonar_basic_encoder", 
        tokenizer="text_sonar_basic_encoder",
        device=device
    )
    print("Text embedder initialized")
except Exception as e:
    print(f"Error loading models: {e}")

## Testing Sentence Similarity with BLASER 2.0

Let's test the sentence similarity functionality with some examples in different languages.

In [ ]:
# Example 1: French to English translation evaluation
src_text = ["Le chat s'assit sur le tapis."]
ref_text = ["The cat sat on the mat."]
mt_text = ["The cat sat down on the carpet."]

print("Source (French):", src_text[0])
print("Reference (English):", ref_text[0])
print("Machine Translation (English):", mt_text[0])

# Get embeddings
src_embs = text_embedder.predict(src_text, source_lang="fra_Latn")
ref_embs = text_embedder.predict(ref_text, source_lang="eng_Latn")
mt_embs = text_embedder.predict(mt_text, source_lang="eng_Latn")

# Predict similarity scores
with torch.inference_mode():
    # Using reference-based model
    ref_score = blaser_ref(src=src_embs, ref=ref_embs, mt=mt_embs).item()
    
    # Using quality estimation model (no reference)
    qe_score = blaser_qe(src=src_embs, mt=mt_embs).item()

print("\nBLASER 2.0 Similarity Scores (scale 1-5, higher is more similar):")
print(f"Reference-based score: {ref_score:.3f}")
print(f"Quality Estimation score: {qe_score:.3f}")

In [ ]:
# Example 2: Testing with multiple language pairs
language_examples = [
    {
        "src": ["Es ist ein schöner Tag heute."], 
        "src_lang": "deu_Latn",
        "ref": ["It is a beautiful day today."],
        "ref_lang": "eng_Latn",
        "mt": ["It's a nice day today."],
        "mt_lang": "eng_Latn",
        "description": "German to English"
    },
    {
        "src": ["El libro está sobre la mesa."], 
        "src_lang": "spa_Latn",
        "ref": ["The book is on the table."],
        "ref_lang": "eng_Latn",
        "mt": ["A book is placed on the table."],
        "mt_lang": "eng_Latn",
        "description": "Spanish to English"
    },
    {
        "src": ["I love studying languages."], 
        "src_lang": "eng_Latn",
        "ref": ["J'adore étudier les langues."],
        "ref_lang": "fra_Latn",
        "mt": ["J'aime apprendre des langues."],
        "mt_lang": "fra_Latn",
        "description": "English to French"
    }
]

print("\n===== Testing Multiple Language Pairs =====\n")

for example in language_examples:
    print(f"\n----- {example['description']} -----")
    print(f"Source ({example['src_lang']}): {example['src'][0]}")
    print(f"Reference ({example['ref_lang']}): {example['ref'][0]}")
    print(f"Machine Translation ({example['mt_lang']}): {example['mt'][0]}")
    
    # Get embeddings
    src_embs = text_embedder.predict(example['src'], source_lang=example['src_lang'])
    ref_embs = text_embedder.predict(example['ref'], source_lang=example['ref_lang'])
    mt_embs = text_embedder.predict(example['mt'], source_lang=example['mt_lang'])
    
    # Predict similarity scores
    with torch.inference_mode():
        ref_score = blaser_ref(src=src_embs, ref=ref_embs, mt=mt_embs).item()
        qe_score = blaser_qe(src=src_embs, mt=mt_embs).item()
        
    print(f"Reference-based score: {ref_score:.3f}")
    print(f"Quality Estimation score: {qe_score:.3f}")

## Evaluating Translations of Varying Quality

Let's test BLASER 2.0 with translations of varying quality to see how well it captures translation quality differences.

In [ ]:
# Example 3: Translations of varying quality
source = ["This research paper presents a novel approach to natural language processing."]
source_lang = "eng_Latn"

translations = [
    {
        "text": ["Cet article de recherche présente une approche novatrice du traitement du langage naturel."],
        "lang": "fra_Latn",
        "quality": "High quality"
    },
    {
        "text": ["Ce papier de recherche présente une nouvelle approche pour le traitement du langage naturel."],
        "lang": "fra_Latn",
        "quality": "Medium quality"
    },
    {
        "text": ["Ce document recherche montre nouveau façon pour traitement langue naturelle."],
        "lang": "fra_Latn",
        "quality": "Low quality"
    },
    {
        "text": ["Ce texte parle de cuisine française et de recettes traditionnelles."],
        "lang": "fra_Latn",
        "quality": "Unrelated content"
    }
]

print(f"\n===== Evaluating Translations of Varying Quality =====\n")
print(f"Source (English): {source[0]}")

# Get source embedding
src_embs = text_embedder.predict(source, source_lang=source_lang)

# Evaluate each translation
for translation in translations:
    print(f"\n{translation['quality']} translation ({translation['lang']}): {translation['text'][0]}")
    
    # Get translation embedding
    mt_embs = text_embedder.predict(translation['text'], source_lang=translation['lang'])
    
    # Predict quality estimation score
    with torch.inference_mode():
        qe_score = blaser_qe(src=src_embs, mt=mt_embs).item()
        
    print(f"Quality Estimation score: {qe_score:.3f}")

## Interactive Text Similarity Evaluation

The following cell allows you to input custom text for similarity evaluation.

In [ ]:
# Custom text input
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create input widgets
src_lang_input = widgets.Text(value='eng_Latn', description='Source Lang:')
src_text_input = widgets.Textarea(value='The weather is nice today.', description='Source Text:')
mt_lang_input = widgets.Text(value='fra_Latn', description='Trans Lang:')
mt_text_input = widgets.Textarea(value='Le temps est beau aujourd\'hui.', description='Translation:')

# Function to evaluate similarity
def evaluate_similarity(button):
    clear_output(wait=True)
    
    # Display inputs
    display(src_lang_input, src_text_input, mt_lang_input, mt_text_input, evaluate_button)
    
    print("\nEvaluating similarity...\n")
    
    src_text = [src_text_input.value]
    mt_text = [mt_text_input.value]
    src_lang = src_lang_input.value
    mt_lang = mt_lang_input.value
    
    try:
        # Get embeddings
        src_embs = text_embedder.predict(src_text, source_lang=src_lang)
        mt_embs = text_embedder.predict(mt_text, source_lang=mt_lang)
        
        # Predict similarity score
        with torch.inference_mode():
            qe_score = blaser_qe(src=src_embs, mt=mt_embs).item()
            
        print(f"Source ({src_lang}): {src_text[0]}")
        print(f"Translation ({mt_lang}): {mt_text[0]}")
        print(f"\nQuality Estimation score: {qe_score:.3f} (scale: 1-5, higher is more similar)")
    except Exception as e:
        print(f"Error: {e}")

# Create evaluate button with callback
evaluate_button = widgets.Button(description="Evaluate Similarity")
evaluate_button.on_click(evaluate_similarity)

# Display widgets
display(src_lang_input, src_text_input, mt_lang_input, mt_text_input, evaluate_button)

## Conclusion

BLASER 2.0 provides a powerful way to evaluate translation quality and sentence similarity across languages using the SONAR embedding space. It can be used in various scenarios:

- Machine translation quality evaluation
- Cross-lingual similarity assessment
- Comparing translation alternatives
- Evaluating speech-to-text translations

For more information, visit:
- [SONAR GitHub Repository](https://github.com/facebookresearch/SONAR)
- [BLASER 2.0 QE Model Card](https://huggingface.co/facebook/blaser-2.0-qe)
- [BLASER 2.0 Ref Model Card](https://huggingface.co/facebook/blaser-2.0-ref)